# Model Verification (1/12)
Check that our base model is working with test data

In [23]:
from pyscipopt import Model, quicksum
import numpy as np
import pandas as pd

In [24]:
m = Model("Verification")

In [25]:
# Read in data
first = pd.read_csv("TestFirstChoice.csv")
second = pd.read_csv("TestSecondChoice.csv")
third = pd.read_csv("TestThirdChoice.csv")

In [26]:
# Get set of students
students = first["Student"]
STUDENTS = {}
for i in range(len(students)):
    STUDENTS[i] = students[i]

In [49]:
# Get set of courses
courses = first.columns
COURSES = {}
for i in range(1,len(courses)):
    COURSES[i] = courses[i]

In [31]:
# Extract preferences
num_students = len(STUDENTS)
num_courses = len(COURSES)
schedule1 = np.zeros([num_students, num_courses])
for i in range(num_students):
    schedule1[i] = first.iloc[i,1:].tolist()
    
schedule2 = np.zeros([num_students, num_courses])
for i in range(num_students):
    schedule2[i] = second.iloc[i,1:].tolist()
    
schedule3 = np.zeros([num_students, num_courses])
for i in range(num_students):
    schedule3[i] = third.iloc[i,1:].tolist()

In [37]:
# Create Variables
X = {}
for i in range(len(STUDENTS)):
    for j in [1,2,3]:
        name = STUDENTS[i] + " pref " + str(j)
        X[i,j] = m.addVar(name, vtype='B')

In [41]:
# Objective
m.setObjective(quicksum(3*X[s,1] + 2*X[s,2] + X[s,1] for s in STUDENTS), "maximize")

In [43]:
# Assignment Constraint (only assigned to one schedule)
for s in range(len(STUDENTS)):
    m.addCons(X[s,1] + X[s,2] + X[s,3] == 1)

In [54]:
# Set Max Course Sizes and add constraint
MAX = [2,2,2,2,2,2]
for c in range(len(COURSES)):
    m.addCons(quicksum(X[s,1]*schedule1[s,c] + X[s,2]*schedule2[s,c] + X[s,3]*schedule3[s,c]
                       for s in range(len(STUDENTS))) <= MAX[c])

In [55]:
m.optimize()

In [63]:
Results = {}
if m.getStatus() == "optimal":
    print("Found Optimal Solution:")
    for i in range(len(STUDENTS)):
        for j in [1,2,3]:
            v = m.getVal(X[i,j])
            if v == 1:
                print(STUDENTS[i], "assigned to choice", str(j))
                Results[STUDENTS[i]] = j
    print(Results)

else:
    print("Did not find optimal solution")
    print("Problem is", m.getStatus())

Found Optimal Solution:
Student 1 assigned to choice 1
Student 2 assigned to choice 1
Student 3 assigned to choice 2
Student 4 assigned to choice 1
{'Student 1': 1, 'Student 2': 1, 'Student 3': 2, 'Student 4': 1}


In [60]:
Results

{'Student 1': 1, 'Student 2': 1, 'Student 3': 2, 'Student 4': 1}

In [73]:
assignment_dict = {1:schedule1, 2:schedule2, 3:schedule3}
enrollments = [0,0,0,0,0,0]
for s in Results:
    snum = int(s[-1]) -1# row number in schedule table
    assignment = Results[s] # points to which schedule 
    enrollments = np.add(enrollments, assignment_dict[assignment][snum])
    

In [71]:
enrollments

array([ 2.,  1.,  1.,  2.,  2.,  0.])